In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf
import numpy as np
import os, shutil


2022-08-01 11:46:23.835031: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 11:46:23.842250: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
%cd /home/guest/fruitfly/

/home/guest/fruitfly


In [5]:
class_names = ['attempt_copulate', 'following', 'licking', 'wing_song']

In [10]:
shutil.rmtree("./test/")
for i in class_names:
    os.makedirs(f'./test/{i}')

In [7]:
my_model = tf.keras.models.load_model("./best_model.h5")
def test_im(arr):
     # Convert single image to a batch.
    input_arr = np.expand_dims(arr, 0)
    predictions = my_model(input_arr, training=False)
    predictions = predictions.numpy()
    classes = np.argmax(predictions, axis = -1)
    print("===>>", class_names[classes[0]])
    print(predictions[0])
    return class_names[classes[0]], predictions[0][classes[0]]

2022-08-01 11:46:40.328219: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-01 11:46:40.328257: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-01 11:46:40.328288: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (guest-desktop): /proc/driver/nvidia/version does not exist


In [8]:
#Code to find the median frame and the absolute difference with the current frame
import numpy as np
import cv2
from math import ceil, floor
import random
from PIL import Image
import os
from glob import glob
import csv
from sys import exit

#padding to make all images equal square shape
desired_size = 80
def padd(arr, box):
    (x, y, w, h) = box
    extra_x = (desired_size-w)/2
    extra_y = (desired_size-h)/2
    extra_x1 = floor(extra_x)
    extra_x2 = ceil(extra_x)
    extra_y1 = floor(extra_y)
    extra_y2 = ceil(extra_y)
    new_im = orig[y1+y-extra_y1:y1+y+h+extra_y2, x1+x-extra_x1:x1+x+w+extra_x2]
    return new_im

video_fn = "./PC1Cam2__21919558__20220517_150136518.avi"
vs = cv2.VideoCapture(video_fn)
#get frames per second
fps = vs.get(cv2.CAP_PROP_FPS)

#calculate the median frame
###############################################################
med_list = []

for i in range(11):
    f_no = random.randint(1, vs.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f_no)
    vs.set(1, f_no)
    ret, frame = vs.read()
    fr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #fr = cv2.GaussianBlur(fr, (21, 21), 0)
    med_list.append(fr)
    
med_arr = np.array(med_list, dtype='uint8')
med_frame = np.median(med_arr, axis=0).astype('uint8')
med_frame = cv2.GaussianBlur(med_frame, (21, 21), 0)
################################################################

# use selectROIs to select only the wells
r =[]
cv2.namedWindow("Crop", cv2.WINDOW_NORMAL)
r = cv2.selectROIs("Crop", frame)
cv2.destroyWindow('Crop')

149
1109
1359
1842
1596
345
2324
1965
984
2136
955


In [12]:
dict_well = {0:'A', 1:'B', 2:'C', 3:'D'}
f = open('time_frames.csv', mode='w')
writer = csv.writer(f)

#vs.set(cv2.CAP_PROP_POS_FRAMES, 0)
vs = cv2.VideoCapture(video_fn)

#########iterate over each well############
for i in range(len(r)):   
    x1, y1, wid, hig = r[i]
    medCrop = med_frame[r[i][1]:r[i][1]+r[i][3], r[i][0]:r[i][0]+r[i][2]]
    name = dict_well[i]
    vs = cv2.VideoCapture(video_fn)
    while True:
        # grab the current frame     
        frame = vs.read()[1]
        # if the frame could not be grabbed, then we have reached the end
        # of the video
        if frame is None:
            break
            
        orig = frame.copy()

        frame_no = int(vs.get(cv2.CAP_PROP_POS_FRAMES)) 
        cur_sec = frame_no//fps
        cur_min = int(cur_sec//60), int(cur_sec%60)
        cur_min = str(cur_min[0])+':'+str(cur_min[1])
        
        frame = frame[r[i][1]:r[i][1]+r[i][3], r[i][0]:r[i][0]+r[i][2]]
        
        # convert it to grayscale, and blur it
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
        # compute the absolute difference between the current frame and
        # median frame
        frameDelta = cv2.absdiff(medCrop, gray)
        thresh = cv2.threshold(frameDelta, 35, 255, cv2.THRESH_BINARY)[1]
        # dilate the thresholded image to fill in holes, then find contours
        # on thresholded image
        thresh = cv2.dilate(thresh, None, iterations=3)
        
        cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[0]
        #sort the contours in the order of decreasing area
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        
        #masked = cv2.bitwise_and(frame,frame, mask=thresh)
        if len(cnts)==1 or cv2.contourArea(cnts[1])<.3*cv2.contourArea(cnts[0]):
            box = cv2.boundingRect(cnts[0])            
            cv2.rectangle(frame, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), (0, 255, 0), 2)
            padded = padd(frame, box)
            sub_class, pred_val = test_im(padded)
            if pred_val<0:
                continue
            cv2.putText(frame, 'well-'+name+': '+sub_class, (5, 15),\
               cv2.FONT_HERSHEY_SIMPLEX, 0.4 , (0,255,0))
            cv2.imwrite(f"./test/{sub_class}/Pos{name}_{int(cur_sec)}_{frame_no}.png", 
                    padded)
            if frame_no%fps == 0:
                writer.writerow(['well-'+name, cur_min, sub_class, pred_val])
        
    
        cv2.namedWindow("Security Feed", cv2.WINDOW_NORMAL)
        cv2.imshow("Security Feed",\
                   cv2.hconcat([frame, cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)]))
        key = cv2.waitKey(1) & 0xFF
        # if the `q` key is pressed, break from the loop
        if key == ord("q") :
            break
        elif key == ord('p'):
            cv2.waitKey(0)
        elif key==27 :
            cv2.destroyAllWindows()
            exit(0)
    
# cleanup the camera and close any open windows
vs.release()
cv2.destroyAllWindows()
f.close()

===>> wing_song
[-2.4318085  0.5586605 -2.2161787  1.2073431]
===>> following
[-1.7923657  1.0196937 -2.0370808 -0.1521402]
===>> following
[-2.6215243   1.7807463  -2.5059783   0.63130623]
===>> following
[-1.6819626  1.0554849 -1.874337  -0.3351964]
===>> wing_song
[-2.9042284  -0.29879034 -0.9029214   2.3001103 ]
===>> following
[-2.7131524   0.99254787 -1.6918116   0.05257308]
===>> following
[-2.205107    0.12081868 -1.0933753  -0.27130488]
===>> following
[-2.5341766   0.5309775  -1.7233276   0.15495609]
===>> following
[-2.4393826  1.1379478 -2.3735409 -0.187436 ]
===>> following
[-3.0063698   0.98081356 -1.570067    0.83228934]
===>> wing_song
[-2.5551164  0.8600247 -2.1606336  1.0574012]
===>> following
[-1.7000493  1.7546949 -0.9984814 -1.4996867]
===>> following
[-1.5807728  1.8190283 -0.767075  -1.9224705]
===>> following
[-2.2222614  2.7299714 -1.9702162 -0.8658402]
===>> wing_song
[-3.2836485   0.61387175 -0.7655745   1.833511  ]
===>> wing_song
[-4.2762156  1.8457954 -2.

===>> attempt_copulate
[ 3.8119795  -3.2222679  -0.40908533 -4.705853  ]
===>> attempt_copulate
[ 1.828205   -2.0072386  -0.01904787 -4.3159    ]
===>> attempt_copulate
[ 0.00778521 -0.6791941  -0.30337474 -2.944099  ]
===>> attempt_copulate
[ 0.11834139 -0.7855588  -0.26459724 -2.7682838 ]
===>> following
[-0.8096145  -0.15790157 -0.65784    -1.8907083 ]
===>> following
[-1.7341709   0.16108604 -0.5370536  -0.9808206 ]
===>> following
[-0.1438486   0.8247548  -0.84375656 -3.3406425 ]
===>> following
[-0.7565605   0.44513962 -0.6136587  -1.7013111 ]
===>> attempt_copulate
[ 4.115018  -3.2940168  1.7184839 -6.8063936]
===>> attempt_copulate
[ 4.51146   -3.7556343  1.7602789 -6.6195726]
===>> attempt_copulate
[ 4.1574655 -3.9690905  2.147743  -6.30332  ]
===>> attempt_copulate
[ 4.275643  -4.457201   2.46804   -6.3965807]
===>> attempt_copulate
[ 4.333178  -4.5370345  1.8303986 -5.4499555]
===>> attempt_copulate
[ 4.4477262 -4.131544   1.5994625 -6.219886 ]
===>> attempt_copulate
[ 4.279

===>> licking
[-1.4487329 -2.5678883  0.8423504  0.6115779]
===>> attempt_copulate
[ 2.7012343 -4.994101   2.2693717 -3.6970308]
===>> attempt_copulate
[ 5.8087196 -6.3267274  1.8777106 -5.564472 ]
===>> attempt_copulate
[ 6.1634626 -6.121855   1.9030671 -6.387326 ]
===>> attempt_copulate
[ 5.8999987 -5.500397   1.0083303 -5.828061 ]
===>> attempt_copulate
[ 5.154152   -4.1617794  -0.02548017 -5.492932  ]
===>> attempt_copulate
[ 5.4614015  -4.408491   -0.30314508 -5.446491  ]
===>> attempt_copulate
[ 4.2129602 -3.4165645  0.1709507 -5.376376 ]
===>> attempt_copulate
[ 3.5349038  -2.8952968   0.25271645 -5.048958  ]
===>> attempt_copulate
[ 2.965215  -1.9715257 -0.3300901 -4.466609 ]
===>> attempt_copulate
[ 2.0443103  -1.6124457   0.50404805 -4.9597597 ]
===>> attempt_copulate
[ 1.3732245  -1.6038048   0.73177034 -4.0703506 ]
===>> attempt_copulate
[ 1.2035364 -1.8048079  1.1706864 -4.099815 ]
===>> attempt_copulate
[ 1.2528913 -1.7662333  1.1026025 -4.019174 ]
===>> attempt_copulate


===>> attempt_copulate
[ 5.9529    -5.3758245  1.4565157 -6.814425 ]
===>> attempt_copulate
[ 5.7323666 -5.286875   1.4183297 -6.4486604]
===>> attempt_copulate
[ 5.685469  -5.140198   1.5025151 -6.6456747]
===>> attempt_copulate
[ 5.387588  -4.994085   1.4694259 -6.164817 ]
===>> attempt_copulate
[ 5.621034  -5.088792   1.3876278 -6.512528 ]
===>> attempt_copulate
[ 5.474363  -5.0600176  1.4529073 -6.2954745]
===>> attempt_copulate
[ 5.531704  -4.8399177  1.7091    -7.066905 ]
===>> attempt_copulate
[ 5.1814322 -4.517871   2.010895  -7.158568 ]
===>> attempt_copulate
[ 5.193101  -4.4974275  1.6633826 -6.934234 ]
===>> attempt_copulate
[ 5.1413956 -4.5278287  1.7423315 -6.9308767]
===>> attempt_copulate
[ 5.086597  -4.415464   1.6570729 -6.8141155]
===>> attempt_copulate
[ 5.1548    -4.4450264  1.7814119 -7.0650206]
===>> attempt_copulate
[ 5.3345323 -4.3827877  1.5736932 -7.3529634]
===>> attempt_copulate
[ 5.350984  -4.4375553  1.6143702 -7.3802376]
===>> attempt_copulate
[ 5.3678513

===>> attempt_copulate
[ 4.736885   -3.4925508   0.57317716 -5.8292413 ]
===>> attempt_copulate
[ 4.7626495  -3.56197     0.56134313 -5.853786  ]
===>> attempt_copulate
[ 4.8141193  -3.5753908   0.59953094 -5.847459  ]
===>> attempt_copulate
[ 5.00683    -3.684487    0.57152367 -6.049569  ]
===>> attempt_copulate
[ 4.6926413  -3.5186064   0.44345087 -5.596763  ]
===>> attempt_copulate
[ 4.6128774  -3.550849    0.58059305 -5.792578  ]
===>> attempt_copulate
[ 4.558512  -3.532043   0.55331   -5.9942894]
===>> attempt_copulate
[ 2.1892223  -1.867029    0.27914146 -4.1802425 ]
===>> following
[-0.23613428  0.11150234 -0.6030976  -1.8244597 ]
===>> following
[-0.45747048  0.07555711 -0.4801614  -1.5234011 ]
===>> following
[-3.3407855  2.0525093 -1.5573336  0.6358623]
===>> licking
[-0.352472    0.02120094  0.5443249  -2.6197064 ]
===>> following
[-0.9845223   0.21088098  0.16764164 -1.4586232 ]
===>> licking
[-0.17209853 -0.01691061  0.46600598 -2.7204764 ]
===>> licking
[-0.36139432 -0.12

...

===>> following
[-2.4389367   1.5822484  -1.3047377  -0.11995096]
===>> following
[-2.0633626   1.480578   -1.3932335  -0.05039782]
===>> following
[-1.6595446   1.1173307  -1.2064621   0.05609927]
===>> following
[-0.9489855   0.6412423  -0.77751106 -0.9611929 ]
===>> following
[-0.97748536  0.6875025  -0.9294328  -0.7811454 ]
===>> following
[-0.95077723  0.5645827  -0.7640082  -0.87055904]
===>> following
[-1.1299219   0.64140004 -0.8039714  -0.6980665 ]
===>> following
[-1.0832744   0.86397207 -1.0009557  -0.6746918 ]
===>> following
[-1.2534854   0.7941923  -0.73136127 -0.719383  ]
===>> following
[-1.1151522  0.739312  -0.8730743 -0.6409191]
===>> following
[-1.0483427   0.73885834 -0.8123866  -0.7885304 ]
===>> following
[-1.141434    0.71172136 -0.72232217 -0.7108587 ]
===>> following
[-1.0480944   0.63432986 -0.6281913  -0.8960168 ]
===>> following
[-0.7400019   0.3353337  -0.49049813 -1.2132862 ]
===>> following
[-0.51282007  0.28422952 -0.61366516 -1.1854879 ]
===>> followin

===>> attempt_copulate
[ 5.205736  -3.7365372  0.5616124 -6.5922985]
===>> attempt_copulate
[ 5.2616887  -3.6653204   0.83686256 -6.869722  ]
===>> attempt_copulate
[ 5.601689   -4.007128    0.49846932 -6.9431844 ]
===>> attempt_copulate
[ 5.637225   -3.983142    0.47563365 -6.917816  ]
===>> attempt_copulate
[ 5.522165   -3.86211     0.56473184 -6.919672  ]
===>> attempt_copulate
[ 5.223437   -3.6203988   0.47890028 -6.490722  ]
===>> attempt_copulate
[ 5.2898574 -3.7018352  0.5529364 -6.6070642]
===>> attempt_copulate
[ 5.3791156  -3.7370503   0.43719557 -6.623567  ]
===>> attempt_copulate
[ 5.154233  -3.633044   0.5309572 -6.4736333]
===>> attempt_copulate
[ 5.1776104 -3.6466844  0.4706197 -6.4539204]
===>> attempt_copulate
[ 4.6815853  -3.5444558   0.35759068 -5.652363  ]
===>> attempt_copulate
[ 5.0380855 -3.6115973  0.6277915 -6.394676 ]
===>> attempt_copulate
[ 4.5797405 -3.470454   0.3749029 -5.562932 ]
===>> attempt_copulate
[ 4.511206   -3.2642124   0.02652804 -5.244233  ]
==

===>> following
[-1.7414484   1.2076864  -0.15704744 -1.3017825 ]
===>> following
[-1.2587337   1.2728926  -0.51099056 -1.5888709 ]
===>> following
[-0.9243327   1.2425307  -0.71480125 -1.7918694 ]
===>> attempt_copulate
[ 1.1855367 -1.3591188  1.1434166 -3.8416657]
===>> attempt_copulate
[ 4.04111   -3.0225585  1.3765976 -6.2420225]
===>> attempt_copulate
[ 3.8304672 -3.3824158  1.9027125 -5.554917 ]
===>> attempt_copulate
[ 4.5594816 -3.687045   2.022769  -6.4377103]
===>> attempt_copulate
[ 4.9283137 -3.913771   2.0565944 -7.0088353]
===>> attempt_copulate
[ 4.873619  -3.929825   2.0981412 -6.981592 ]
===>> attempt_copulate
[ 4.3263297 -3.0676324  1.7247969 -6.9342527]
===>> attempt_copulate
[ 3.5124183 -2.6411204  1.3178958 -5.828572 ]
===>> attempt_copulate
[ 2.063259  -2.106915   1.1476982 -4.0549035]
===>> attempt_copulate
[ 1.2696321 -1.7626059  1.2139928 -3.416052 ]
===>> licking
[ 1.3026224 -1.8183692  1.3157139 -3.5214908]
===>> licking
[ 1.2335746 -1.9207455  1.3697003 -3.3

===>> attempt_copulate
[ 0.42815313 -1.117215    0.04946104 -1.7410489 ]
===>> attempt_copulate
[ 0.27677432 -0.6824307  -0.14566003 -1.7347716 ]
===>> attempt_copulate
[ 0.3449797  -0.6092382  -0.34153235 -1.6195985 ]
===>> attempt_copulate
[ 0.22409128 -0.5297308  -0.37906742 -1.4812151 ]
===>> attempt_copulate
[-0.04248982 -0.30108884 -0.43254742 -1.3306803 ]
===>> attempt_copulate
[-0.08150233 -0.2740584  -0.3628662  -1.3780376 ]
===>> attempt_copulate
[ 0.3222244  -0.79995304 -0.13623075 -1.700631  ]
===>> attempt_copulate
[ 0.1390241 -0.2714452 -0.4597817 -1.8066963]
===>> attempt_copulate
[ 0.07344139 -0.12030108 -0.4997631  -1.8806741 ]
===>> attempt_copulate
[ 0.5361018 -0.6531766 -0.5812962 -1.6921234]
===>> attempt_copulate
[ 0.94410473 -0.9887029  -0.48440823 -1.942974  ]
===>> attempt_copulate
[ 0.9668949  -0.87135893 -0.7229456  -1.9739028 ]
===>> attempt_copulate
[ 1.0045124  -0.83410335 -0.72612256 -1.8703654 ]
===>> attempt_copulate
[ 1.0008157 -0.768575  -0.8222123 -1

===>> wing_song
[-1.4528018  -0.61584663 -0.66814744  0.31960863]
===>> attempt_copulate
[ 0.34710547 -1.2033616  -0.3025942  -2.3395894 ]
===>> attempt_copulate
[ 4.0796824  -3.5455499   0.63790464 -5.7531486 ]
===>> attempt_copulate
[ 5.486589  -4.642107   0.7104364 -6.129568 ]
===>> attempt_copulate
[ 5.1555295 -4.9013906  1.4967644 -5.7044544]
===>> attempt_copulate
[ 5.240461  -5.378823   1.7181519 -5.717997 ]
===>> attempt_copulate
[ 5.566941  -4.8126535  0.8011209 -5.9603863]
===>> attempt_copulate
[ 6.183184  -4.7491455 -0.0315783 -6.408308 ]
===>> attempt_copulate
[ 6.157228   -4.7253222  -0.19797073 -6.26909   ]
===>> attempt_copulate
[ 5.623971   -4.109063   -0.08747739 -5.9077044 ]
===>> attempt_copulate
[ 4.3023143  -3.1761942  -0.56314874 -4.815864  ]
===>> attempt_copulate
[ 3.3009799  -2.4376025  -0.71519077 -4.403171  ]
===>> attempt_copulate
[ 2.4822772 -2.097549  -0.5259857 -4.0629563]
===>> attempt_copulate
[ 1.9173079  -1.658794   -0.74387395 -3.6404104 ]
===>> att

In [13]:
for fln in os.listdir('Cutouts/'):
    sub_class, pred_val = test_im(os.path.join("Cutouts/", fln))
    if pred_val<0:
        continue
    shutil.copy(os.path.join("Cutouts/", fln), f'test/{sub_class}')
    break

1/1 [==============================] - 0s 140ms/step
Cutouts/Crpp_14768_0.png ===>> wing_song
[-3.3914218  -0.15914679 -2.0753925   2.586185  ]
(1, 80, 80, 3)
